In [1]:
#!pip install selenium

In [2]:


from bs4 import BeautifulSoup


# import webdriver to initialise a browser
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys

##############################################################

#extracting the content
from html_table_extractor.extractor import Extractor 

import requests
import lxml.html as lh
import pandas as pd
import numpy as np


#website scraping from (NJSAMS)
url = ("https://njsams.rutgers.edu/njsams/Reports/SummaryReport/StateSummaryReportMenu.aspx")

#initilaize webdriver and put the path where download the driver
driver = webdriver.Chrome('/Users/brandenkushnir/Desktop/NJSAMS_Project/chromedriver')

# Launch Chrome and pass the url
driver.get(url)

In [3]:
#Get the specific county
county = Select(driver.find_element_by_id('ctl00_cphMainContent_rptMenu_ddlCounties'))


    #NJ State
#NJ = county.select_by_value('0')





    #Burlington
#burlington = county.select_by_value('03')

    #Gloucester
#gloucester = county.select_by_value('08')

    #Mercer
#mercer = county.select_by_value('11')

    #Middlesex
#middlesex = county.select_by_value('12')

    #Monmouth
#monmouth = county.select_by_value('13')

    #Ocean
ocean = county.select_by_value('15')







In [4]:
##Dates that I want to extract the data from

In [5]:
#start date
startPath = '//*[@id="ctl00_cphMainContent_rptMenu_txtFromDate"]'

datebox = driver.find_element_by_xpath(startPath)
#01/1/2015
#01/1/2016
#01/1/2017
#01/1/2018
#01/1/2019

datebox.send_keys("01/1/2019")


In [6]:
#end date
endPath = '//*[@id="ctl00_cphMainContent_rptMenu_txtToDate"]'

datebox = driver.find_element_by_xpath(endPath)

#01/1/2016
#01/1/2017
#01/1/2018
#01/1/2019
#8/25/2020
datebox.send_keys("8/25/2020")

In [7]:
#open child window and find its location

firstWindow = driver.window_handles

parent = driver.current_window_handle


button = driver.find_element_by_id('ctl00_cphMainContent_rptMenu_btnGetReport')

button.click()
WebDriverWait(driver, 10).until(
    EC.number_of_windows_to_be(2)
    )   



##driver window is now the child window
child = driver.window_handles[1]      
driver.switch_to.window(child)


#print ("Parent Window ID is : %s" %firstWindow)
#
#print ("Child Window ID is : %s" %child)
#print("Child Window Title is : %s " %(driver.title)) 


In [8]:
#pip install html-table-extractor
#pip install pyspark

In [9]:


# 5 different tables in the web driver
tb1 = driver.find_element_by_xpath("/html/body/form/table[3]/tbody/tr[2]/td[1]").get_attribute('outerHTML')

tb2 = driver.find_element_by_xpath("/html/body/form/table[3]/tbody/tr[2]/td[2]").get_attribute('outerHTML')




#months table with all three
tb3 = driver.find_element_by_xpath("/html/body/form/table[5]/tbody/tr[1]/td[1]").get_attribute('outerHTML')

#first months table
#tb3 = driver.find_element_by_xpath("/html/body/form/table[5]/tbody/tr[1]").get_attribute('outerHTML')


#second months tables
tb4 = driver.find_element_by_xpath("/html/body/form/table[5]/tbody/tr[1]/td[2]").get_attribute('outerHTML')

#3 months table
tb5 = driver.find_element_by_xpath("/html/body/form/table[5]/tbody/tr[1]/td[3]").get_attribute('outerHTML')



'''
Here I am trying to to get the table information 
however the type of dataframe is a list

'''
#df1 = pd.read_html(tb1,skiprows=[0,7,25,37], columns=['Topic','Total','Percent Mode'])


#print(type(df1))
#df2 = pd.read_html(tb2,skiprows=[0,3,12,18,22,27,32],header=0)
#print(df2)
#df3 = pd.read_html(tb3,flavor='bs4')
#print(df3)
#df4 = pd.read_html(tb4)
#df5 = pd.read_html(tb5)

###################################################


####created table 1 dataframe
soup = BeautifulSoup(tb1, 'html.parser')
extractor = Extractor(soup, id_='ucAdmissionCntrl_tblReportPart1')
extractor.parse()
extractor.return_list()
extractor.write_to_csv(path='.')


header_list= ["Topic","Total","Percent Mode"]
data = pd.read_csv("output.csv",names=header_list,skiprows=[0,7,25,37])
#data

In [10]:
####created table 2 dataframe
soup1 = BeautifulSoup(tb2, 'html.parser')
extractor = Extractor(soup1, id_='ucAdmissionCntrl_tblReportPart2')
extractor.parse()
extractor.return_list()
extractor.write_to_csv(path='.')


header_list= ["Topic","Total","Percent Mode"]
data1 = pd.read_csv("output.csv",names=header_list,skiprows=[0,3,12,13,19,23,28,33])
#data1



###created table 3
soup2 = BeautifulSoup(tb3, 'html.parser')
extractor = Extractor(soup2, id_='ucAdmissionCntrl_tblReportMonth1')
extractor.parse()
extractor.return_list()
extractor.write_to_csv(path='.')


header_list= ["Topic","Total","Percent Mode"]
data2 = pd.read_csv("output.csv",names=header_list,skiprows=[])
#data2

####created table 4
soup3 = BeautifulSoup(tb4, 'html.parser')
extractor = Extractor(soup3, id_='ucAdmissionCntrl_tblReportMonth2')
extractor.parse()
extractor.return_list()
extractor.write_to_csv(path='.')


header_list= ["Topic","Total","Percent Mode"]
data3 = pd.read_csv("output.csv",names=header_list,skiprows=[])
#data3

####created table 5 (LAST TABLE)
soup4 = BeautifulSoup(tb5, 'html.parser')
extractor = Extractor(soup4, id_='ucAdmissionCntrl_tblReportMonth3')
extractor.parse()
extractor.return_list()
extractor.write_to_csv(path='.')


header_list= ["Topic","Total","Percent Mode"]
data4 = pd.read_csv("output.csv",names=header_list,skiprows=[])
#data4




In [14]:
#################################################


#merged data
merged_df = pd.concat([data,data1,data2,data3,data4], ignore_index=True)
merged_df_tail = merged_df.tail(50)
merged_df

############################################

#creating keys from data


            #PD            #LevelofCare       Referral Sourrce
frames  = [merged_df[0:6], merged_df[6:23],   merged_df[23:33],   
           
           #Prior Care          #Race/Ethnicity        #sed
           merged_df[33:34],  merged_df[34:39],      merged_df[39:41],
           
           #Age                #Employemnt Status   #Education
           merged_df[41:49],   merged_df[49:54],   merged_df[54:57],
          
          #Living            #Smoke tobacco      #Legal Problem
         merged_df[57:60],  merged_df[60:61],    merged_df[61:65],
          
          #Insurance       #Same County treatment      #Months
        merged_df[65:70],  merged_df[70:71],           merged_df[71:83]]


df_keys = pd.concat(frames, keys=['Primary Drug','Level of Care',
                                  'Referral Source','Prior Care',
                                  'Race/Ethnicity','Sex','Age',
                                  'Employment Status','Education',
                                  'Living','Smoke Tobacco','Legal Problem',
                                  'Insurance', 'In County Treatment','Months'])


merged_df = df_keys
merged_df
#df_keys
#############################
#'''Data has finally been organized and is ready to be examined'''

Topic  Total  \
Primary Drug 0   \n                        Alcohol\n           ...   4301   
             1   \n                        Heroin & Other Opiat...   6372   
             2   \n                        Cocaine/Crack\n     ...    594   
             3   \n                        Marijuana\n         ...    988   
             4   \n                        Other Drugs\n       ...    511   
...                                                            ...    ...   
Months       78  \n                        August\n            ...   1113   
             79  \n                        September\n         ...    649   
             80  \n                        October\n           ...    742   
             81  \n                        November\n          ...    654   
             82  \n                        December\n          ...    639   

                Percent Mode  
Primary Drug 0         33.6%  
             1         49.8%  
             2          4.6%  
             3          7.7%  
             4          4.0%  
...                      ...  
Months       78         8.7%  
             79         5.1%  
             80         5.8%  
             81         5.1%  
             82         5.0%  

[83 rows x 3 columns]

In [15]:


#removes  all /n which was an unwanted string carried from the webscrape
merged_df['Topic'] = merged_df['Topic'].str.replace(r'\n', '')

#convert percentage
merged_df['Percent Mode'] = merged_df['Percent Mode'].str.replace(r'%', '')

merged_df

Topic  Total  \
Primary Drug 0                           Alcohol               ...   4301   
             1                           Heroin & Other Opiates...   6372   
             2                           Cocaine/Crack         ...    594   
             3                           Marijuana             ...    988   
             4                           Other Drugs           ...    511   
...                                                            ...    ...   
Months       78                          August                ...   1113   
             79                          September             ...    649   
             80                          October               ...    742   
             81                          November              ...    654   
             82                          December              ...    639   

                Percent Mode  
Primary Drug 0          33.6  
             1          49.8  
             2           4.6  
             3           7.7  
             4           4.0  
...                      ...  
Months       78          8.7  
             79          5.1  
             80          5.8  
             81          5.1  
             82          5.0  

[83 rows x 3 columns]

**This is 2015-2016 Burlington**

In [ ]:
#merged_df.to_csv("2015-2016_B.csv")

#B15_16 = pd.read_csv('2015-2016_B.csv')
#B15_16 = B15_16.drop(columns=['Unnamed: 1']).rename(columns={"Unnamed: 0":"Subject"})
#B15_16




**This is 2016-2017 Burlington**

In [ ]:
#merged_df.to_csv("2016-2017_B.csv")


#B16_17 = pd.read_csv('2016-2017_B.csv')
#B16_17 = B16_17.drop(columns=['Unnamed: 1']).rename(columns={"Unnamed: 0":"Subject"})
#B16_17

**This is 2017-2018 Burlington**

In [ ]:
#merged_df.to_csv("2017-2018_B.csv")


#B17_18 = pd.read_csv('2017-2018_B.csv')
#B17_18 = B17_18.drop(columns=['Unnamed: 1']).rename(columns={"Unnamed: 0":"Subject"})
#B17_18

**This is 2018-2019 Burlington**

In [ ]:
#merged_df.to_csv("2018-2019_B.csv")


#B18_19 = pd.read_csv('2018-2019_B.csv')
#B18_19
#B18_19 = B18_19.drop(columns=['Unnamed: 1']).rename(columns={"Unnamed: 0":"Subject"})
#B18_19

**This is 2019-2020**

In [ ]:
##Soonest possible day is 8/25/2020

#merged_df.to_csv("2019-2020_B.csv")


#B19_20 = pd.read_csv('2019-2020_B.csv')

#B19_20 = B19_20.drop(columns=['Unnamed: 1']).rename(columns={"Unnamed: 0":"Subject"})
#B19_20

**All Years combined**

In [ ]:
#merged_df.to_csv("allYears_B.csv")


#B15_20 = pd.read_csv('allYears_B.csv')

#B15_20 = B15_20.drop(columns=['Unnamed: 1']).rename(columns={"Unnamed: 0":"Subject"})
#B15_20

**The above methods will be how I will get the data from all counties**